In [1]:
import os
import json
import psycopg2
from psycopg2 import sql


## Patients data from the downloads patients record JSON file

In [11]:
# Database connection details
DB_CONFIG = {
     "dbname": "PADS_sql",
    "user": "postgres",
    "password": "eysk123",
    "host": "localhost",
    "port": 5432
}

# Folder containing JSON files
FOLDER_PATH = r"C:\Users\akals\Downloads\pads-parkinsons-disease-smartwatch-dataset-1.0.0\pads-parkinsons-disease-smartwatch-dataset-1.0.0\patients"

def import_json_to_postgres():
    conn = None
    cursor = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Loop through all files in the folder
        for filename in os.listdir(FOLDER_PATH):
            if filename.endswith(".json"):
                file_path = os.path.join(FOLDER_PATH, filename)

                # Read JSON file
                with open(file_path, 'r') as file:
                    data = json.load(file)

                    # Prepare insert query using psycopg2.sql module for dynamic table and column names
                    query = sql.SQL("""
                    INSERT INTO patient_data (
                        resource_type, id, study_id, condition, disease_comment,
                        age_at_diagnosis, age, height, weight, gender, handedness,
                        appearance_in_kinship, appearance_in_first_grade_kinship,
                        effect_of_alcohol_on_tremor
                    ) VALUES (
                        %(resource_type)s, %(id)s, %(study_id)s, %(condition)s, %(disease_comment)s,
                        %(age_at_diagnosis)s, %(age)s, %(height)s, %(weight)s, %(gender)s, %(handedness)s,
                        %(appearance_in_kinship)s, %(appearance_in_first_grade_kinship)s,
                        %(effect_of_alcohol_on_tremor)s
                    ) ON CONFLICT (id) DO NOTHING;
                    """)

                    cursor.execute(query, {
                        'resource_type': data['resource_type'],
                        'id': data['id'],
                        'study_id': data['study_id'],
                        'condition': data['condition'],
                        'disease_comment': data['disease_comment'],
                        'age_at_diagnosis': data['age_at_diagnosis'],
                        'age': data['age'],
                        'height': data['height'],
                        'weight': data['weight'],
                        'gender': data['gender'],
                        'handedness': data['handedness'],
                        'appearance_in_kinship': data['appearance_in_kinship'],
                        'appearance_in_first_grade_kinship': data['appearance_in_first_grade_kinship'],
                        'effect_of_alcohol_on_tremor': data['effect_of_alcohol_on_tremor']
                    })

        # Commit changes
        conn.commit()
        print("Data imported successfully.")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Run the import function
if __name__ == "__main__":
    import_json_to_postgres()


Data imported successfully.


## questionnarie response from JSON to postgres table questionnarie responses

In [49]:
# Database connection details
DB_CONFIG = {
     "dbname": "PADS_sql",
    "user": "postgres",
    "password": "eysk123",
    "host": "localhost",
    "port": 5432
}

# Specify the folder name containing JSON files (in the same directory as the script)
folder_name = "questionnaire"

# Get the absolute path of the folder
folder_path = os.path.join(os.getcwd(), folder_name)

def insert_data_to_postgres(records):
    conn = None
    cursor = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Insert records into PostgreSQL
        query = """
        INSERT INTO questionnaire_responses (
            resource_type, subject_id, study_id, questionnaire_id, questionnaire_name, link_id, question, answer
        ) VALUES (
            %(resource_type)s, %(subject_id)s, %(study_id)s, %(questionnaire_id)s, %(questionnaire_name)s, %(link_id)s, %(question)s, %(answer)s
        );
        """
        
        cursor.executemany(query, records)

        # Commit changes
        conn.commit()
        print("Data inserted successfully.")

    except psycopg2.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def process_json_files(folder_path):
    # List to store all records
    all_records = []

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):  # Ensure it's a JSON file
            file_path = os.path.join(folder_path, file_name)
            
            # Open and load JSON data
            with open(file_path, 'r') as f:
                data = json.load(f)
                
                # Extract the common metadata
                resource_type = data.get("resource_type", "")
                subject_id = data.get("subject_id", "")
                study_id = data.get("study_id", "")
                questionnaire_id = data.get("id", "")
                questionnaire_name = data.get("questionnaire_name", "")
                
                # Extract the items (questions and answers)
                for item in data.get("item", []):
                    link_id = item.get("link_id", "")
                    text = item.get("text", "")
                    answer = item.get("answer", False)  # Default to False if not present
                    
                    # Append a record to the list
                    all_records.append({
                        "resource_type": resource_type,
                        "subject_id": subject_id,
                        "study_id": study_id,
                        "questionnaire_id": questionnaire_id,
                        "questionnaire_name": questionnaire_name,
                        "link_id": link_id,
                        "question": text,
                        "answer": answer
                    })

    return all_records

if __name__ == "__main__":
    records = process_json_files(folder_path)
    insert_data_to_postgres(records)


Data inserted successfully.


## Movement data from JSON to postgres table movement with the extracton time series text document 

In [ ]:
# Base directories
base_folder = "movement"
timeseries_folder = os.path.join(base_folder, "timeseries")
database_name = "PADS_sql"
user = "postgres"
password = "eysk123"
host = "localhost"
port = "5432"

# Create the PostgreSQL database connection and table
conn = psycopg2.connect(
    dbname=database_name, user=user, password=password, host=host, port=port
)
cursor = conn.cursor()

# Create the table (adjust columns as necessary)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS movement_data (
        id SERIAL PRIMARY KEY,
        subject_id TEXT,
        study_id TEXT,
        device_id TEXT,
        record_id TEXT,
        record_name TEXT,
        rows INTEGER,
        device_location TEXT,
        time FLOAT,
        accelerometer_x FLOAT,
        accelerometer_y FLOAT,
        accelerometer_z FLOAT,
        gyroscope_x FLOAT,
        gyroscope_y FLOAT,
        gyroscope_z FLOAT
    )
''')

# Traverse through all JSON files
for file_name in os.listdir(base_folder):
    if file_name.endswith(".json"):
        json_path = os.path.join(base_folder, file_name)
        
        with open(json_path, "r") as json_file:
            record = json.load(json_file)
            
            # Extract metadata
            subject_id = record.get("subject_id")
            study_id = record.get("study_id")
            device_id = record.get("device_id")
            record_id = record.get("id")
            
            # Iterate through each session
            for session in record.get("session", []):
                record_name = session.get("record_name")
                rows = session.get("rows")
                
                # Iterate through each record
                for rec in session.get("records", []):
                    device_location = rec.get("device_location")
                    file_name = rec.get("file_name")
                    
                    # Construct the path to the timeseries file
                    timeseries_path = os.path.join(base_folder, file_name)
                    
                    if os.path.exists(timeseries_path):
                        # Read the timeseries file
                        with open(timeseries_path, "r") as ts_file:
                            for line in ts_file:
                                values = line.strip().split(",")  # Split by comma
                                
                                # Insert data into the database
                                cursor.execute('''
                                    INSERT INTO movement_data (
                                        subject_id, study_id, device_id, record_id,
                                        record_name, rows, device_location, time, accelerometer_x,
                                        accelerometer_y, accelerometer_z, gyroscope_x, gyroscope_y, gyroscope_z
                                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                ''', (
                                    subject_id, study_id, device_id, record_id,
                                    record_name, rows, device_location,
                                    float(values[0]), float(values[1]), float(values[2]), float(values[3]),
                                    float(values[4]), float(values[5]), float(values[6])
                                ))
                    else:
                        print(f"File not found: {timeseries_path}")

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Processed data saved to the PostgreSQL database.")